In [ ]:
# -------------------------------
# Imports
# -------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from pymongo import MongoClient
from dotenv import load_dotenv
import os

# -------------------------------
# Plotting style
# -------------------------------
sns.set(style="whitegrid", palette="muted", font_scale=1.2)
plt.rcParams["figure.figsize"] = (12,6)
